In [1]:
import numpy as np

In [2]:
# 1D
a = np.ones(5)
b = np.ones(5)

a.dot(b).shape

()

In [3]:
# 2D
a = np.ones((5, 6))
b = np.ones((6, 7))

a.dot(b).shape

(5, 7)

In [4]:
# 2D x 1D
a = np.ones((5, 6))
b = np.ones((6))

a.dot(b).shape

(5,)

In [5]:
# 3D x 2D
a = np.ones((2,3,4))
b = np.ones((4, 5))

a.dot(b).shape

(2, 3, 5)

In [6]:
# 3D
a = np.ones((2,3,4))
b = np.ones((6,4,7))

a.dot(b).shape

(2, 3, 6, 7)

In [7]:
def dsl_ops(array1: np.ndarray, array2: np.ndarray):
    return np.concatenate((array1, array2))

In [8]:
dsl_ops(np.ones((3, 50, 50)), np.ones((3, 50, 50))).shape

(6, 50, 50)

In [9]:
np.ones((3, 50, 50)).sum(1).shape

(3, 50)

1D arrays- equal to mul


2D arrays- equivalent to matmul


We need two things to work for dot product- multiplication of arrays, and sum of arrays.


In [10]:
def dsl_sum_op(array1: np.ndarray, target_shape: tuple):
    """ Ideally we could use this when using an array slice """
    return array1.reshape((-1, *target_shape))

In [11]:
dsl_sum_op(np.ones((3, 50, 50)), (3, 50)).shape

(50, 3, 50)

In [12]:
dsl = np.random.choice(np.arange(7), (10, 10))
dsl

array([[0, 0, 6, 2, 2, 3, 2, 1, 4, 0],
       [3, 1, 0, 3, 2, 5, 4, 6, 5, 4],
       [4, 1, 4, 5, 4, 4, 3, 6, 1, 2],
       [5, 2, 6, 6, 3, 2, 1, 1, 5, 4],
       [0, 1, 4, 6, 4, 6, 1, 4, 0, 4],
       [3, 2, 3, 0, 3, 4, 1, 5, 1, 5],
       [4, 6, 3, 1, 3, 6, 6, 3, 0, 3],
       [5, 6, 0, 6, 2, 2, 6, 1, 2, 6],
       [6, 6, 3, 0, 3, 5, 2, 0, 6, 6],
       [2, 4, 2, 0, 4, 1, 4, 5, 1, 2]])

In [13]:
# Sum of first row, second row, etc
dsl.sum(axis=1), dsl.sum(axis=1).shape

(array([20, 33, 34, 35, 30, 27, 35, 36, 37, 25]), (10,))

In [14]:
dsl_sum_op(dsl, dsl.sum(axis=1).shape).shape

(10, 10)

In [15]:
dsl_sum_op(dsl, dsl.sum().shape).shape, dsl.sum().shape

((100,), ())

In [16]:
from syft import PhiTensor, DataSubjectList

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
pt = PhiTensor(
    child=np.ones((50, 50)), 
    data_subjects=DataSubjectList.from_objs(np.random.choice(np.arange(100), (3, 50, 50))), 
    min_vals=0, 
    max_vals=1
)

In [18]:
DataSubjectList.from_objs(np.random.choice(np.arange(100), (3, 50, 50)))

In [19]:
pt.data_subjects

In [20]:
res = pt.sum()

In [21]:
res.data_subjects.shape

(7500,)

In [22]:
res.shape

()

In [23]:
res = pt.sum(axis=1)

In [24]:
res.data_subjects.shape

(150, 50)

In [25]:
res.shape

(50,)

In [26]:
gt = pt.gamma
gt.shape

(50, 50)

In [27]:
res = gt.sum()

()


In [28]:
res.child.shape

()

In [29]:
res.data_subjects.shape

(7500,)

In [30]:
res.shape

()

In [31]:
res = pt.gamma.sum(axis=1)

(50,)


In [32]:
res.data_subjects.shape

(150, 50)

In [33]:
res.shape

(50,)

In [ ]:
def dot(dsl1: DataSubjectList, dsl2: DataSubjectList, target_shape):
    final_shape = 

In [ ]:
def multiplication(dsl1: DataSubjectList, dsl2: DataSubjectList):
    output_ds = DataSubjectList(
        one_hot_lookup=np.array([]),
        data_subjects_indexed=np.empty(())